# Common Data Format Conversion
Every of our datasources keep their metadata in different formats and have different sets of parameters. In this notebook, we'll join them all into a proper .csv file for the future data processing and learning.

### Constants and imports

In [1]:
import pandas as pd
import json
import os

METADATA_DIRECTORY = 'metadata'
CMA_METADATA_FILE = os.path.join(METADATA_DIRECTORY, 'cma.json')
MET_METADATA_FILE = os.path.join(METADATA_DIRECTORY, 'met.json')
NGA_METADATA_FILE = os.path.join(METADATA_DIRECTORY, 'nga.json')
RESULT_METADATA_FILE = os.path.join(METADATA_DIRECTORY, 'metadata_joined.csv')

### Preparing output dataframe

In [2]:
result = pd.DataFrame()

### Loading CMA data

In [3]:
if os.path.exists(CMA_METADATA_FILE):
    with open(CMA_METADATA_FILE, 'r') as f:
        cma_metadata = json.load(f)

In [4]:
if cma_metadata is not None:
    rows = []
    for art_id in cma_metadata:
        row = {}
        metadata = cma_metadata[art_id]
        row['id'] = f'cma_{art_id}'
        row['title'] = metadata['title']
        row['date'] = metadata['creation_date']
        row['begin_date'] = None
        row['end_date'] = None
        row['culture'] = metadata['culture']
        row['technique'] = metadata['technique']
        row['type'] = metadata['type']
        row['department'] = metadata['department']
        row['collection'] = metadata['collection']
        row['path'] = metadata['path']
        rows.append(row)
        
    df_cma = pd.DataFrame(rows)
    result = pd.concat([result, df_cma], ignore_index=True)

In [5]:
result.describe()

,id,title,date,begin_date,end_date,culture,technique,type,department,collection,path
count,32614,32614,31258,0,0,32614,32318,32614,32614,32614,32614
unique,32614,23344,4872,0,0,4340,6385,61,19,163,32614
top,cma_122190,Page from Tales of a Parrot (Tuti-nama): text ...,1800s,NaN,NaN,"France, 19th century",etching,Print,Prints,PR - Etching,dataset/raw/cma_122190.jpg
freq,1,366,925,NaN,NaN,3064,2174,8536,8398,3146,1


### Loading MET data

In [6]:
if os.path.exists(MET_METADATA_FILE):
    with open(MET_METADATA_FILE, 'r') as f:
        met_metadata = json.load(f)

In [7]:
if met_metadata is not None:
    rows = []
    for art_id in met_metadata:
        row = {}
        metadata = met_metadata[art_id]
        if 'path' not in metadata:
            continue
        row['id'] = f'met_{art_id}'
        row['title'] = metadata['objectName']
        row['date'] = metadata['objectDate']
        row['begin_date'] = metadata['objectBeginDate']
        row['end_date'] = metadata['objectEndDate']
        row['culture'] = f"{metadata['culture']}, {metadata['period']}"
        row['technique'] = metadata['medium']
        row['type'] = metadata['classification']
        row['department'] = metadata['department']
        row['collection'] = None
        row['path'] = metadata['path']
        rows.append(row)
        
    df_met = pd.DataFrame(rows)
    result = pd.concat([result, df_met], ignore_index=True)

In [8]:
result.describe()

,id,title,date,begin_date,end_date,culture,technique,type,department,collection,path
count,44854,44854,43498,12240,12240,44854,44558,44854,44854,32614,44854
unique,44854,25613,7706,874,873,5298,9887,376,35,163,44811
top,cma_122190,Print,1800s,1700,1899,",",etching,Print,Prints,PR - Etching,dataset/raw/met_696662.jpg
freq,1,1928,925,676,610,6025,2175,8536,8398,3146,5


### Loading NGA data

In [9]:
if os.path.exists(NGA_METADATA_FILE):
    with open(NGA_METADATA_FILE, 'r') as f:
        nga_metadata = json.load(f)

In [10]:
if nga_metadata is not None:
    rows = []
    for art_id in nga_metadata:
        row = {}
        metadata = nga_metadata[art_id]
        if 'path' not in metadata:
            continue
        row['id'] = f'met_{art_id}'
        row['title'] = metadata['title']
        row['date'] = metadata['displaydate']
        row['begin_date'] = metadata['beginyear']
        row['end_date'] = metadata['endyear']
        row['culture'] = metadata['terms']['School'] if 'School' in metadata['terms'] else None
        row['technique'] =  metadata['terms']['Technique'] if 'Technique' in metadata['terms'] else None
        row['type'] = metadata['classification']
        row['department'] = None
        row['collection'] = metadata['creditline']
        row['path'] = metadata['path']
        rows.append(row)
        
    df_nga = pd.DataFrame(rows)
    result = pd.concat([result, df_nga], ignore_index=True)

In [11]:
result.describe()

,id,title,date,begin_date,end_date,culture,technique,type,department,collection,path
count,74826,74826,70938,42174.0,42172.0,71744,56045,74826,44854,62581,74826
unique,74168,48719,10774,1025.0,999.0,5439,10036,381,35,1928,74783
top,met_224273,Print,c. 1936,1936.0,1936.0,American,gelatin silver print,Print,Prints,Index of American Design,dataset/raw/met_696662.jpg
freq,2,1928,1071,1346.0,1316.0,15678,3441,21264,8398,4885,5


### Saving the .csv file

In [12]:
result.to_csv(RESULT_METADATA_FILE, index=False)